In [39]:
# Updated 2024.11.02
import pandas as pd
import requests
import csv
from datetime import datetime, timedelta

# Your API Key from The Odds API
API_KEY = 'Insert API Key Here'

# Define constants for the API call
SPORT = 'mma_mixed_martial_arts'  # MMA sport key
REGIONS = 'us'  # Focus on US market
MARKETS = 'h2h'  # Only fetch head-to-head markets
ODDS_FORMAT = 'american'  # American odds format
DATE_FORMAT = 'iso'  # ISO date format for timestamps

# Base URL for the Odds API
BASE_URL = 'https://api.the-odds-api.com/v4/historical/sports'

# Function to get event dates between a start and end date with 12-hour offset
def get_event_dates_between(masterlist_df, start_date, end_date):
    """
    This function filters the masterlist for event dates between the specified start and end dates,
    applies a 12-hour offset, and returns the list of unique event dates.
    """
    # Convert input dates to datetime objects
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Filter masterlist for event dates within the given range
    filtered_masterlist = masterlist_df[(masterlist_df['event_date'] >= start_date) & (masterlist_df['event_date'] <= end_date)].copy()
    
    # Apply a 12-hour offset to the event dates using .loc[] to avoid the warning
    filtered_masterlist.loc[:, 'event_date'] = filtered_masterlist['event_date'] + pd.Timedelta(hours=12)
    
    # Extract unique event dates
    event_dates = filtered_masterlist['event_date'].unique()
    
    # Convert event dates back to a list and return
    return list(event_dates), len(event_dates)

# Function to get historical MMA odds for a specific timestamp
def get_historical_mma_odds(api_key, snapshot_date):
    """
    Function to get historical MMA odds for a specific timestamp.
    """
    url = f'{BASE_URL}/{SPORT}/odds'
    params = {
        'api_key': api_key,
        'regions': REGIONS,
        'markets': MARKETS,
        'date': snapshot_date,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT
    }
    
    # Make the API request
    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        print(f'Failed to get historical odds: status_code {response.status_code}, response body {response.text}')
        return None, response

    # Return the parsed JSON response and the response object to access headers
    return response.json(), response

# Function to save odds data to a CSV file
def save_odds_to_csv(odds_data, file_name='mma_odds_(2024-08-24 to 2020-06-06).csv'):
    """
    Function to save odds data to a CSV file with utf-8 encoding to handle special characters.
    """
    with open(file_name, mode='a', newline='', encoding='utf-8') as file:  # Specify utf-8 encoding
        writer = csv.writer(file)
        
        # Write header if the file is empty
        if file.tell() == 0:
            writer.writerow(['Timestamp', 'Home Team', 'Away Team', 'Commence Time', 'Bookmaker', 'Market', 'Outcome Name', 'Odds Price'])
        
        # Write odds data to the CSV file
        for event in odds_data['data']:
            for bookmaker in event['bookmakers']:
                for market in bookmaker['markets']:
                    for outcome in market['outcomes']:
                        writer.writerow([odds_data['timestamp'], 
                                         event['home_team'], 
                                         event['away_team'], 
                                         event['commence_time'], 
                                         bookmaker['title'], 
                                         market['key'], 
                                         outcome['name'], 
                                         outcome['price']])


# Function to iterate through event dates and get odds for each timestamp
def get_and_save_odds_for_dates(api_key, event_dates, file_name='mma_odds_(2024-08-24 to 2020-06-06).csv', limit=None):
    """
    Function to iterate through event dates and retrieve odds, saving them to a CSV file.
    
    Parameters:
    - api_key: The Odds API key
    - event_dates: A list of event dates with 12-hour offset
    - file_name: Name of the CSV file to save the data to
    - limit: Maximum number of timestamps to process (set to None to process all dates)
    """
    if limit:
        event_dates = event_dates[:limit]  # Apply limit if specified
    
    for i, event_date in enumerate(event_dates):  # Iterate over all timestamps if limit is None
        snapshot_date = event_date.strftime("%Y-%m-%dT%H:%M:%SZ")
        
        print(f"\nFetching odds for timestamp: {snapshot_date}")
        odds_data, response = get_historical_mma_odds(api_key, snapshot_date)
        
        if odds_data:
            # Save odds data to CSV
            save_odds_to_csv(odds_data, file_name)
            print(f"Odds saved for timestamp: {snapshot_date}")
        else:
            print(f"No data available for timestamp {snapshot_date}")

        # Print the remaining and used requests after each API call
        print(f"\nRemaining requests: {response.headers.get('x-requests-remaining', 'N/A')}")
        print(f"Used requests: {response.headers.get('x-requests-used', 'N/A')}")

def main():
    # Load masterlist CSV with date parsing
    masterlist_file = r"C:\Users\EditZ\event_masterlist (updated 09-06-2024).csv"
    masterlist_df = pd.read_csv(masterlist_file, encoding='utf-8', parse_dates=['event_date'])

    # Define date range for filtering
    start_date = '2020-06-06'
    end_date = '2024-08-24'

    # Get event dates within the range
    event_dates_list, num_events = get_event_dates_between(masterlist_df, start_date, end_date)
    print(f"There are {num_events} events between {start_date} and {end_date}.")
    print(f"Event dates (with 12-hour offset): {event_dates_list}")

    # Generate a dynamic file name based on the date range
    file_name = f'mma_odds_({start_date} to {end_date}).csv'

    # Retrieve and save odds data for the first 5 event dates
    get_and_save_odds_for_dates(API_KEY, event_dates_list, file_name=file_name, limit=5)
 

if __name__ == "__main__":
    main()


There are 176 events between 2020-06-06 and 2024-06-08.
Event dates (with 12-hour offset): [Timestamp('2024-06-08 12:00:00'), Timestamp('2024-06-01 12:00:00'), Timestamp('2024-05-18 12:00:00'), Timestamp('2024-05-11 12:00:00'), Timestamp('2024-05-04 12:00:00'), Timestamp('2024-04-27 12:00:00'), Timestamp('2024-04-13 12:00:00'), Timestamp('2024-04-06 12:00:00'), Timestamp('2024-03-30 12:00:00'), Timestamp('2024-03-23 12:00:00'), Timestamp('2024-03-16 12:00:00'), Timestamp('2024-03-09 12:00:00'), Timestamp('2024-03-02 12:00:00'), Timestamp('2024-02-24 12:00:00'), Timestamp('2024-02-17 12:00:00'), Timestamp('2024-02-10 12:00:00'), Timestamp('2024-02-03 12:00:00'), Timestamp('2024-01-20 12:00:00'), Timestamp('2024-01-13 12:00:00'), Timestamp('2023-12-16 12:00:00'), Timestamp('2023-12-09 12:00:00'), Timestamp('2023-12-02 12:00:00'), Timestamp('2023-11-18 12:00:00'), Timestamp('2023-11-11 12:00:00'), Timestamp('2023-11-04 12:00:00'), Timestamp('2023-10-21 12:00:00'), Timestamp('2023-10-14 12

In [2]:
import csv
from collections import defaultdict
from statistics import mean

# Define a function to load odds data from the CSV file and structure it for easy display
def load_odds_from_csv(file_name):
    fights_with_odds = defaultdict(lambda: {
        'Event Date': None,
        'Fighter A': None,
        'Fighter B': None,
        'Fighter A Odds': [],
        'Fighter B Odds': [],
        'Fighter A Avg Odds': None,
        'Fighter B Avg Odds': None,
        'Fighter A Best Odds': None,
        'Fighter B Best Odds': None
    })
    
    # Read odds data from CSV
    with open(file_name, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            event_key = (row['Home Team'], row['Away Team'], row['Commence Time'])
            
            # Initialize event information if not already set
            if not fights_with_odds[event_key]['Event Date']:
                fights_with_odds[event_key]['Event Date'] = row['Commence Time']
                fights_with_odds[event_key]['Fighter A'] = row['Home Team']
                fights_with_odds[event_key]['Fighter B'] = row['Away Team']
            
            # Collect odds data for each fighter
            odds_info = {
                'Bookmaker': row['Bookmaker'],
                'Odds Price': float(row['Odds Price'])
            }
            if row['Outcome Name'] == fights_with_odds[event_key]['Fighter A']:
                fights_with_odds[event_key]['Fighter A Odds'].append(odds_info)
            elif row['Outcome Name'] == fights_with_odds[event_key]['Fighter B']:
                fights_with_odds[event_key]['Fighter B Odds'].append(odds_info)
    
    # Calculate average and best odds for each fighter
    for fight, details in fights_with_odds.items():
        if details['Fighter A Odds']:
            odds_prices = [odds['Odds Price'] for odds in details['Fighter A Odds']]
            details['Fighter A Avg Odds'] = mean(odds_prices)
            details['Fighter A Best Odds'] = min(details['Fighter A Odds'], key=lambda x: x['Odds Price'])
        
        if details['Fighter B Odds']:
            odds_prices = [odds['Odds Price'] for odds in details['Fighter B Odds']]
            details['Fighter B Avg Odds'] = mean(odds_prices)
            details['Fighter B Best Odds'] = min(details['Fighter B Odds'], key=lambda x: x['Odds Price'])
    
    return fights_with_odds

# Display function to format and print the structured odds data
def display_fights_with_odds(fights_with_odds):
    for event_key, fight in fights_with_odds.items():
        print(f"Event Date: {fight['Event Date']}")
        print(f"Fighter A: {fight['Fighter A']}")
        
        # Fighter A odds information
        if fight['Fighter A Odds']:
            print("  Fighter A Odds:")
            for odds in fight['Fighter A Odds']:
                print(f"    Bookmaker: {odds['Bookmaker']}, Odds Price: {odds['Odds Price']}")
            print(f"  Fighter A Avg Odds: {fight['Fighter A Avg Odds']}")
            print(f"  Fighter A Best Odds: {fight['Fighter A Best Odds']['Odds Price']} (Bookmaker: {fight['Fighter A Best Odds']['Bookmaker']})")
        else:
            print("  No odds available for Fighter A")
        
        print(f"\nFighter B: {fight['Fighter B']}")
        
        # Fighter B odds information
        if fight['Fighter B Odds']:
            print("  Fighter B Odds:")
            for odds in fight['Fighter B Odds']:
                print(f"    Bookmaker: {odds['Bookmaker']}, Odds Price: {odds['Odds Price']}")
            print(f"  Fighter B Avg Odds: {fight['Fighter B Avg Odds']}")
            print(f"  Fighter B Best Odds: {fight['Fighter B Best Odds']['Odds Price']} (Bookmaker: {fight['Fighter B Best Odds']['Bookmaker']})")
        else:
            print("  No odds available for Fighter B")
        
        print("\n" + "=" * 50 + "\n")

# Load odds from the CSV and display them
file_name = r"C:\Users\EditZ\UFC Research\New Pipeline\(DONT TOUCH) mma_odds_(2024-08-24 to 2020-06-06).csv"
fights_with_odds = load_odds_from_csv(file_name)
display_fights_with_odds(fights_with_odds)


Event Date: 2024-08-24T17:00:00Z
Fighter A: Albert Odzimkowski
  Fighter A Odds:
    Bookmaker: Bovada, Odds Price: -185.0
    Bookmaker: LowVig.ag, Odds Price: -175.0
  Fighter A Avg Odds: -180.0
  Fighter A Best Odds: -185.0 (Bookmaker: Bovada)

Fighter B: David Hosek
  Fighter B Odds:
    Bookmaker: Bovada, Odds Price: 140.0
    Bookmaker: LowVig.ag, Odds Price: 145.0
  Fighter B Avg Odds: 142.5
  Fighter B Best Odds: 140.0 (Bookmaker: Bovada)


Event Date: 2024-08-24T17:00:00Z
Fighter A: Krystian Blezien
  Fighter A Odds:
    Bookmaker: Bovada, Odds Price: -155.0
    Bookmaker: LowVig.ag, Odds Price: -150.0
  Fighter A Avg Odds: -152.5
  Fighter A Best Odds: -155.0 (Bookmaker: Bovada)

Fighter B: Alvin Lowenski
  Fighter B Odds:
    Bookmaker: Bovada, Odds Price: 120.0
    Bookmaker: LowVig.ag, Odds Price: 120.0
  Fighter B Avg Odds: 120.0
  Fighter B Best Odds: 120.0 (Bookmaker: Bovada)


Event Date: 2024-08-24T17:00:00Z
Fighter A: Oleksii Polischuk
  Fighter A Odds:
    Bookmaker

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

